In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,LassoCV,RidgeCV
from sklearn.compose import ColumnTransformer,make_column_selector
from statsmodels.stats.descriptivestats import Description
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

filename = 'SDR 2021 - Database.xlsx'

In [ ]:
df_trend = pd.read_excel(filename,sheet_name='Data for Trends',usecols='B:N')
display(df_trend.info())
display(df_trend.describe())
#plt.figure()
df_trend.groupby('Year').mean().reset_index().drop('Population',axis=1).plot(x='Year',figsize=(16,9))


In [ ]:
X = pd.read_excel(filename,sheet_name='Raw Data',usecols='B:P')
X.drop('Population in 2020',axis=1,inplace=True)
X['Regions used for the SDG Index & Dashboard'] = pd.Series(X['Regions used for the SDG Index & Dashboard'],dtype='category')
Y = pd.read_excel(filename,sheet_name='SDR2021 Data', usecols='B,C')
# No he decidido si eliminarlos o si predecir el score con ellos.
Ys = Y.dropna(subset=['2021 SDG Index Score']) # Se eliminan los que no tienen variable de respuesta por resultar inútiles. 
df = X.merge(Y,on='Country',how='inner').set_index('Country')
valid_values = X.merge(Ys,on='Country',how='inner').set_index('Country')
X.set_index('Country',inplace=True)
Y.set_index('Country',inplace=True)
display(df.info())
display(df.describe())

In [ ]:

groups = df.groupby('Regions used for the SDG Index & Dashboard')
for name, group in groups:
    plt.figure(figsize=(16,9))
    plt.title(f'Matriz de correlación de {name}')
    sns.heatmap(group.corr(),cmap='Spectral')

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr(),cmap='Spectral',annot=True)

In [ ]:
g = sns.pairplot(df.drop('Eswatini'),corner=True,)#kind='reg')
for ax in g.axes.flatten():
   if ax:
       # rotate x axis labels
       ax.set_xlabel(ax.get_xlabel(), rotation = 45)
       # rotate y axis labels
       ax.set_ylabel(ax.get_ylabel(), rotation = -45)
       # set y labels alignment
       ax.yaxis.get_label().set_horizontalalignment('right')

In [ ]:
tr = PowerTransformer()
datum = tr.fit_transform(df.select_dtypes(include=np.number).drop(['Eswatini']))

g = sns.pairplot(pd.DataFrame(datum),corner=True,kind='reg')
for ax in g.axes.flatten():
    if ax:
        # rotate x axis labels
        ax.set_xlabel(ax.get_xlabel(), rotation = 45)
        # rotate y axis labels
        ax.set_ylabel(ax.get_ylabel(), rotation = -45)
        # set y labels alignment
        #ax.yaxis.get_label().set_horizontalalignment('right')

In [ ]:
df.sort_values(by='Prevalence of wasting in children under 5 years of age (%)',ascending=False)

In [ ]:
df['Regions used for the SDG Index & Dashboard'].nunique()

In [ ]:

imputer = Pipeline(
    [
        ('Scaler',PowerTransformer()),
        ('KNN Imputer', KNNImputer(n_neighbors=7)),
#        ('PCA',PCA())
    ]
)

transformer = ColumnTransformer(
    [
        ('Impute and scale', imputer, make_column_selector(dtype_include=np.number)),
        ('Encoder Region',OneHotEncoder(),['Regions used for the SDG Index & Dashboard'])
    ]
)

pipe = Pipeline(
    [
        ('Transform group',transformer),
        ('Ridge regression', RidgeCV(alphas=np.logspace(-4,2,20)))
    ]
)

pipe2 = Pipeline(
    [
        ('Transform group',transformer),
        ('Linear regression', LinearRegression())
    ]
)
pipe3 = Pipeline(
    [
        ('Transform group',transformer),
        ('Lasso regression', LassoCV())
    ]
)

Xes = valid_values.drop('2021 SDG Index Score',axis=1)
Yes = valid_values['2021 SDG Index Score']
pipe.fit(Xes,Yes)
pipe2.fit(Xes,Yes)
pipe3.fit(Xes,Yes)

print(f'Coef Ridge: {pipe.score(Xes,Yes)}')
print(pipe['Ridge regression'].alpha_)
print(pipe['Ridge regression'].coef_)
print(f'Coef Linear: {pipe2.score(Xes,Yes)}')
#print(pipe['Ridge regression'].alpha_)
print(pipe2['Linear regression'].coef_)
print(f'Coef Lasso: {pipe3.score(Xes,Yes)}')
print(pipe3['Lasso regression'].alpha_)
print(pipe3['Lasso regression'].coef_)

### NOTAS
- Los índices `Yield Gap Closure` y `Poverty Rate after Taxes and Transfers` solamente son aplicables a la OCDE, por lo que se eliminan como variables predictoras 

df.dtypes()

In [ ]:
df.drop('Regions used for the SDG Index & Dashboard',1).isna().groupby(df['Regions used for the SDG Index & Dashboard']).sum()

In [ ]:
doubledf = valid_values.drop(['Poverty rate after taxes and transfers (%)','Yield gap closure (% of potential yield)'],axis=1)
xx = doubledf.drop('2021 SDG Index Score',axis=1)
yy = doubledf['2021 SDG Index Score']

trains1 = []
tests1 = []
trains2 = []
tests2 = []
trains3 = []
tests3 = []

for i in range(20):
    xtrain,xtest,ytrain,ytest = train_test_split(xx,yy)
    pipe.fit(xtrain,ytrain)
    pipe2.fit(xtrain,ytrain)
    pipe3.fit(xtrain,ytrain)
    trains1.append(mean_squared_error(ytrain,pipe.predict(xtrain)))
    tests1.append(mean_squared_error(ytest,pipe.predict(xtest)))
    trains2.append(mean_squared_error(ytrain,pipe2.predict(xtrain)))
    tests2.append(mean_squared_error(ytest,pipe2.predict(xtest)))
    trains3.append(mean_squared_error(ytrain,pipe3.predict(xtrain)))
    tests3.append(mean_squared_error(ytest,pipe3.predict(xtest)))

eps1 = pipe.predict(xtrain)-ytrain
eps2 = pipe2.predict(xtrain)-ytrain
eps3 = pipe3.predict(xtrain)-ytrain

residuales = pd.DataFrame({
        'Residuales Ridge': eps1, 
        'Residuales OLS': eps2, 
        'Residuales Lasso': eps3, 
    })

residuales.plot.kde()
plt.grid()
plt.title('Distribución de residuales de SDG Index Score de modelos')

print(f'Train MSE Ridge: {np.mean(trains1):4f}')
print(f'Test MSE Ridge: {np.mean(tests1):4f}')
print(pipe['Ridge regression'].alpha_)

print(f'Train MSE OLS: {np.mean(trains2):4f}')
print(f'Test MSE OLS: {np.mean(tests2):4f}')

print(f'Train MSE Lasso: {np.mean(trains3):4f}')
print(f'Test MSE Lasso: {np.mean(tests3):4f}')
print(pipe3['Lasso regression'].alpha_)

In [ ]:
plt.figure(figsize=(16,9))
sns.kdeplot(data=df[['Regions used for the SDG Index & Dashboard','Human Trophic Level (best 2-3 worst)']].drop('Eswatini'),x='Human Trophic Level (best 2-3 worst)',hue='Regions used for the SDG Index & Dashboard')
plt.grid()
psd = df.select_dtypes(np.number)
plt.figure(figsize=(16,9))
sns.ecdfplot(data=(psd-psd.min())/(psd.max()-psd.min()))
plt.xlabel('Min-max normalized values')
plt.xlim([0,1])
plt.grid()
plt.figure(figsize=(16,9))
sns.ecdfplot(data=pd.DataFrame(datum,columns=df.select_dtypes(np.number).columns))
x = np.linspace(-3,3)
from scipy.stats import norm
normales = norm.cdf(x)
plt.plot(x,normales,color='red')
plt.xlabel('$\sigma$')
plt.grid()

In [ ]:
desc = Description(residuales)
desc.summary()

In [ ]:
sns.kdeplot(trains1)
sns.kdeplot(tests1)
sns.kdeplot(trains2)
sns.kdeplot(tests2)
sns.kdeplot(trains3)
sns.kdeplot(tests3)

In [ ]:
u,d,v = np.linalg.svd(pipe['Transform group'].transform(Xes))

phis = v.T**2/d**2
pis = phis.T/sum(phis.T)
sns.heatmap(pis,cmap='Spectral')
plt.xlabel('Número de variable')
plt.ylabel('Número de condición')
plt.title('Proporciones de la descomposición de varianzas')

In [ ]:
u,d,v = np.linalg.svd(imputer.fit_transform(Xes.select_dtypes(np.number)))
phis = v.T**2/d**2
pis = phis.T/sum(phis.T)
sns.heatmap(pis,cmap='Spectral')
plt.xlabel('Número de variable')
plt.ylabel('Número de condición')
plt.title('Proporciones de la descomposición de varianzas')

In [ ]:
ranks = [min(Yes),max(Yes)]
plt.figure(figsize=(8,6))
plt.scatter(Yes,pipe.predict(Xes),marker='x',linewidths=1)
plt.scatter(Yes,pipe2.predict(Xes),marker='x',linewidths=1)
plt.scatter(Yes,pipe3.predict(Xes),marker='x',linewidths=1)
plt.legend(['Ridge','OLS','Lasso'])
plt.grid()
plt.xlabel('Valores reales')
plt.ylabel('Valores predichos')
plt.plot(ranks,ranks, color='red',alpha=0.7)
plt.title('Predicciones de SDG Index Score')

In [ ]:
ridgedf = pd.DataFrame(data= {
        'SDG True': Yes,
        'SDG Predicted': pipe.predict(Xes),
    })
ridgedf['Model'] = 'Ridge'
OLSdf = pd.DataFrame(data= {
        'SDG True': Yes,
        'SDG Predicted': pipe.predict(Xes),
    })
OLSdf['Model'] = 'OLS'
lassodf = pd.DataFrame(data= {
        'SDG True': Yes,
        'SDG Predicted': pipe.predict(Xes),
    })
lassodf['Model'] = 'Lasso'

models = pd.concat([ridgedf,OLSdf,lassodf]).reset_index()

sns.jointplot(
    data= models,
    x='SDG True',
    y='SDG Predicted',
    hue='Model',
    marker='x'
)

In [ ]:
filtro = Y.isna()['2021 SDG Index Score']
missingSDGs = pd.DataFrame(
    data = {
        'Country': Y[filtro].index,
        'Ridge': pipe.predict(X[filtro]),
        'OLS': pipe2.predict(X[filtro]),
        'Lasso': pipe3.predict(X[filtro]),
    }
)
missingSDGs.to_csv('MissingSDGs.csv',float_format='%.2f')
